In [216]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import numpy as np
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("CarriageKSAAll.csv") 
# Preview the first 5 lines of the loaded data 


All Variables

In [284]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'favorites', 'hashtags', 'id', 'mentions',
       'minute', 'name', 'permalinks', 'retweets', 'text', 'time_sec', 'urls',
       'user_name'],
      dtype='object')

In [285]:
data.shape

(18562, 14)

Remove usernames that are nan

In [287]:
listofauthors = data['user_name']
listofauthors = list(listofauthors)
df = data[~pd.isnull(listofauthors)]

In [288]:
df.shape

(4109, 14)

Keep only three columns username, time_sec, link to tweets

In [293]:
# tweetsdata is a list of tuples
tweetsdata = zip(df['user_name'],df['time_sec'], df['permalinks'])
tweetsdata = [i for i in tweetsdata]

# To get unique askers, we exclude tweets with username "CarriageKSA" or from multilple users in this case it will contain a space

In [294]:
# number of questions
askers = [i[0] for i in tweetsdata if 'CarriageKSA' not in i[0] and ' ' not in i[0]]
# list after set returns unique items in the list
askers = list(set(askers))
len(askers)

318

# Algorithm to extract timing about one person 


In [299]:
# Run the function below first
# for testing the function
person_name_tag = '@Mnoor_ssi' 
company_name_tag = '@CarriageKSA'
get_average_time_to_reply_to_one_person(company_name_tag, person_name_tag)


[('@Mnoor_ssi', 1548158086, 'https://twitter.com/Mnoor_ssi/status/1087679966375038976')]
[('@Mnoor_ssi', 'https://twitter.com/Mnoor_ssi/status/1087679966375038976', 0)]


(-1, 'https://twitter.com/Mnoor_ssi/status/1087679966375038976')

In [ ]:
def get_average_time_to_reply_to_one_person(company_name_tag, person_name_tag):
    # function to study the name
    # get all tweets where the person is tagged
    userdata = [(i,j,id) for (i,j,id) in tweetsdata if person_name_tag in i]
    
    #----------------------------------
    # Algorithm to extract time different 
    # First, get the time of each tweet
    time = [j for i,j,m in userdata]
    
    # Second, shift the time one step and remove the first time
    # here I assume the tweets are arleady sorted, I inspected the data and found them 
    # --- already sorted , if not modify the code
    # to substrate time from previous tweet
    timeshifted = time.copy()
    timeshifted.append(timeshifted[-1])
    timeshifted.pop(0)
    
    # Third, get the time difference
    # assumption the the data are sorted
    timediff = [i- j for i,j in zip(time,timeshifted)]
    
    #print some stuff to get more information for debugging
    #print(userdata)
    #------------------------------------
    # create a list of tuples for the users tags beside it the time different
    # print it to check
    reviewlist = [(i[0],j) for i,j in zip(userdata,timediff)]
    
    # full review list only for debugging with the tweet link
    #fullreviewlist = [(i[0],i[2],j) for i,j in zip(userdata,timediff)]
    #print(fullreviewlist)
    # collect reply times for each tweet started by the user
    
    # loop through the tweets of the person
    replytimes = []
    for x in range(len(reviewlist)):
        i,j = reviewlist[x]
        # check if the person is in the tags if so ...
        if i == person_name_tag:
            # check if the tweet directly above it is from the company
            # here I am looking for the first time the company replies to the user
            # 
            if company_name_tag in reviewlist[x-1][0]:
                # Again, this means the user asked question in x and the company 
                # ---replied in tweet number x - 1 and finds the difference 
                starttime = reviewlist[x][1]
                replytime = reviewlist[x-1][1]
                replytimes.append(replytime)
    if len(replytimes) > 0:
        # get the average of the time spent to reply to the person for the first time
        average_time_to_reply_to_person = np.mean(replytimes)
        return (average_time_to_reply_to_person,'ok')
    else:
        # if it didn't find any reply, instead of returning ok it returns the link 
        # for the tweet to check
        return (-1, userdata[0][2])


# Potential analyses

We can anlyze the text to the tweets that are not replied to , and generate a beautiful
word cloud graph where the decision maker finds out what words dominate the messages that
did not get any reply

# Loop through all the askers and get the same information

Instead of the print below, please append them together in a large dictionary
The requested statistics are now simple , 
1- Number of askers (already answered above)
2- Number of questions without reply (simply count -1 in the dictionary or add a counter in the loop below)
3- Average time to reply (exclude the -1 tweets and get the average of the rest , you can do it also in the loop)

In [300]:
noreply = 0
for person_name_tag in askers:
    
    #person_name_tag = '@coredor3' 
    company_name_tag = '@CarriageKSA'
    # if the tweet didn't get reply from the company, link to the tweet will be attached
    reply_time = get_average_time_to_reply_to_one_person(company_name_tag, person_name_tag)
    if reply_time[0] == -1:
        noreply = noreply + reply_time[0]
    print(person_name_tag,reply_time[0],reply_time[1])

[('@oshom11 @eje1991 @CarriageKSA @McDonaldsKSA', 1558647810, 'https://twitter.com/oshom11/status/1131677054666182658'), ('@eje1991', 1558647753, 'https://twitter.com/eje1991/status/1131676817994211330'), ('@oshom11 @eje1991 @CarriageKSA @McDonaldsKSA', 1558647047, 'https://twitter.com/oshom11/status/1131673857566355476'), ('@eje1991', 1558646962, 'https://twitter.com/eje1991/status/1131673498835922945'), ('@eje1991', 1558646869, 'https://twitter.com/eje1991/status/1131673109357056000')]
[('@oshom11 @eje1991 @CarriageKSA @McDonaldsKSA', 'https://twitter.com/oshom11/status/1131677054666182658', 57), ('@eje1991', 'https://twitter.com/eje1991/status/1131676817994211330', 706), ('@oshom11 @eje1991 @CarriageKSA @McDonaldsKSA', 'https://twitter.com/oshom11/status/1131673857566355476', 85), ('@eje1991', 'https://twitter.com/eje1991/status/1131673498835922945', 93), ('@eje1991', 'https://twitter.com/eje1991/status/1131673109357056000', 0)]
@eje1991 71.0 ok
[('@esraa930923', 1565675893, 'https:

[('@_ays97', 1563901188, 'https://twitter.com/_ays97/status/1153711321378099201')]
[('@_ays97', 'https://twitter.com/_ays97/status/1153711321378099201', 0)]
@_ays97 -1 https://twitter.com/_ays97/status/1153711321378099201
[('@jobs2saudi', 1541820600, 'https://twitter.com/jobs2saudi/status/1061098621607669760'), ('@jobs2saudi', 1541820600, 'https://twitter.com/jobs2saudi/status/1061098621607669760')]
[('@jobs2saudi', 'https://twitter.com/jobs2saudi/status/1061098621607669760', 0), ('@jobs2saudi', 'https://twitter.com/jobs2saudi/status/1061098621607669760', 0)]
@jobs2saudi -1 https://twitter.com/jobs2saudi/status/1061098621607669760
[('@aster3M', 1562658663, 'https://twitter.com/aster3M/status/1148499792751943680')]
[('@aster3M', 'https://twitter.com/aster3M/status/1148499792751943680', 0)]
@aster3M -1 https://twitter.com/aster3M/status/1148499792751943680
[('@muhrahasila100', 1541842223, 'https://twitter.com/muhrahasila100/status/1061189314510491648'), ('@muhrahasila100', 1541842223, 'h

[('@Mnoor_ssi', 1548158086, 'https://twitter.com/Mnoor_ssi/status/1087679966375038976')]
[('@Mnoor_ssi', 'https://twitter.com/Mnoor_ssi/status/1087679966375038976', 0)]


(-1, 'https://twitter.com/Mnoor_ssi/status/1087679966375038976')

@eje1991 71.0 ok
@esraa930923 -1 https://twitter.com/esraa930923/status/1161154971255095297
@qdOzhCWKEHw1Q5l -1 https://twitter.com/qdOzhCWKEHw1Q5l/status/1132062700572479488
@AbdulazizAA99 1560.0 ok
@MaG_3z -1 https://twitter.com/MaG_3z/status/1130189459935383559
@Faisal_Dagh -1 https://twitter.com/Faisal_Dagh/status/1125296579424980993
@5e8ISe9sFdqZRFZ -1 https://twitter.com/5e8ISe9sFdqZRFZ/status/1130489696654680064
@sara_fy6 -1 https://twitter.com/sara_fy6/status/1125133882314166273
@waledalsaif -1 https://twitter.com/waledalsaif/status/1153672051745525760
@Zeee_69 -1 https://twitter.com/Zeee_69/status/1136312615028908032
@plp666 -1 https://twitter.com/plp666/status/1097102030856966145
@Pearl_00_ -1 https://twitter.com/Pearl_00_/status/1140768349607120896
@1samah1412 -1 https://twitter.com/1samah1412/status/1061188930312167424
@Nais_jk -1 https://twitter.com/Nais_jk/status/1145771259273392128
@767AD -1 https://twitter.com/767AD/status/1120564380696023040
@sajoona1406 -1 https://twi

In [271]:
noreply

-277